In [1]:
import pandas as pd
import nivapy3 as nivapy

# Interactive map for "1000 Lakes" survey

Note the following:

 * I'm currently using the site locations from the 1995 survey
 
 * You need to "zip" `index.html` before you download it, otherwise it'll open as a new webpage (right-clicking and using Save-As on this webpage **does not** work!)
 
First, modify `quickmap` to include layers from Kartverket.

In [2]:
def quickmap(df, lon_col='longitude', lat_col='latitude', popup=None, cluster=False, 
             tiles='Stamen Terrain', aerial_imagery=False, kartverket=False, 
             layer_name='Stations'):
    """ Make an interactive map from a point dataset. Can be used with any dataframe
        containing lat/lon co-ordinates (in WGS84 decimal degrees), but primarily 
        designed to be used directly with the functions in nivapy.da.
        
    Args:
        df:            Dataframe. Must include columns for lat and lon in WGS84 decimal degrees
        lon_col:       Str. Column with longitudes
        lat_col:       Str. Column with latitudes
        popup:         Str or None. Default None. Column containing text for popup labels
        cluster:       Bool. Whether to implement marker clustering
        tiles:         Str. Basemap to use. See folium.Map for full details. Choices:        
                            - 'OpenStreetMap'
                            - 'Mapbox Bright' (Limited levels of zoom for free tiles)
                            - 'Mapbox Control Room' (Limited levels of zoom for free tiles)
                            - 'Stamen' (Terrain, Toner, and Watercolor)
                            - 'Cloudmade' (Must pass API key)
                            - 'Mapbox' (Must pass API key)
                            - 'CartoDB' (positron and dark_matter)
                            - Custom tileset by passing a Leaflet-style URL to the tiles
                              parameter e.g. http://{s}.yourtiles.com/{z}/{x}/{y}.png
        aerial_imagery: Bool. Whether to include Google satellite serial imagery as an
                        additional layer
        layer_name:     Str. Name of layer to create in "Table of Contents"                 
        
    Returns:
        Folium map
    """
    import pandas as pd
    import folium
    from folium.plugins import FastMarkerCluster
    from folium.plugins import MarkerCluster

    # Drop NaN
    df2 = df.dropna(subset=[lon_col, lat_col])

    # Get data
    if popup:
        df2 = df2[[lat_col, lon_col, popup]]
        df2[popup] = df2[popup].astype(str)
    else:
        df2 = df2[[lat_col, lon_col]]

    # Setup map
    avg_lon = df2[lon_col].mean()
    avg_lat = df2[lat_col].mean()        
    map1 = folium.Map(location=[avg_lat, avg_lon],
                      zoom_start=4,
                      tiles=tiles)

    # Add aerial imagery if desired
    if aerial_imagery:
        folium.raster_layers.TileLayer(tiles='http://{s}.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
                                       attr='google',
                                       name='Google satellite',
                                       max_zoom=20,
                                       subdomains=['mt0', 'mt1', 'mt2', 'mt3'],
                                       overlay=False,
                                       control=True).add_to(map1)
        
    if kartverket:
        folium.raster_layers.TileLayer(tiles='https://opencache.statkart.no/gatekeeper/gk/gk.open_gmaps?layers=topo4&zoom={z}&x={x}&y={y}',
                                       attr='karkverket',
                                       name='Kartverket topographic',
                                       overlay=False,
                                       control=True).add_to(map1)
        
        folium.raster_layers.TileLayer(tiles='https://opencache.statkart.no/gatekeeper/gk/gk.open_gmaps?layers=terreng_norgeskart&zoom={z}&x={x}&y={y}',
                                       attr='karkverket',
                                       name='Kartverket terrain',
                                       overlay=False,
                                       control=True).add_to(map1)

    folium.raster_layers.WmsTileLayer(url='https://kart.miljodirektoratet.no/arcgis/services/villrein/MapServer/WMSServer',
                                      layers='villrein_leveomrade_grense',
                                      fmt='image/png',
                                      transparent=True,
                                      attr='miljødirektoratet',
                                      name='Villrein leveområde',
                                      overlay=True,
                                      control=True,
                                      show=False).add_to(map1)

    folium.raster_layers.WmsTileLayer(url='https://kart.miljodirektoratet.no/arcgis/services/vern/mapserver/WMSServer',
                                      layers='naturvern_klasser_omrade',
                                      fmt='image/png',
                                      transparent=True,
                                      attr='miljødirektoratet',
                                      name='Naturvern område',
                                      overlay=True,
                                      control=True,
                                      show=False).add_to(map1)
    
    # Add feature group to map
    grp = folium.FeatureGroup(name=layer_name)
    
    # Draw points
    if cluster and popup:
        locs = list(zip(df2[lat_col].values, df2[lon_col].values))
        popups = list(df2[popup].values)

        # Marker cluster with labels
        marker_cluster = MarkerCluster(locations=locs,
                                       popups=popups)        
        grp.add_child(marker_cluster)
        grp.add_to(map1)
       
    elif cluster and not popup:
        locs = list(zip(df2[lat_col].values, df2[lon_col].values))
        marker_cluster = FastMarkerCluster(data=locs)
        grp.add_child(marker_cluster)
        grp.add_to(map1)
        
    elif not cluster and popup: # Plot separate circle markers, with popup    
        for idx, row in df2.iterrows():
            marker = folium.CircleMarker(location=[row[lat_col], row[lon_col]],
                                         radius=5,
                                         weight=1,
                                         color='black',
                                         popup=folium.Popup(row[popup], parse_html=False),
                                         fill_color='red',
                                         fill_opacity=1)
            grp.add_child(marker)
        grp.add_to(map1)

    else: # Plot separate circle markers, no popup    
        for idx, row in df2.iterrows():
            marker = folium.CircleMarker(location=[row[lat_col], row[lon_col]],
                                         radius=5,
                                         weight=1,
                                         color='black',
                                         fill_color='red',
                                         fill_opacity=1)
            grp.add_child(marker)
        grp.add_to(map1)
            
    # Add layer control
    folium.LayerControl().add_to(map1)

    # Zoom to data
    xmin, xmax = df2[lon_col].min(), df2[lon_col].max()
    ymin, ymax = df2[lat_col].min(), df2[lat_col].max()
    map1.fit_bounds([[ymin, xmin], [ymax, xmax]])
        
    return map1

In [3]:
# Read data
df = pd.read_excel(r'../../1000_lakes_2019.xlsx')
df.head()

,index,station_id,station_code,station_name,lake_name,komm_nr,komm,fylke_nr,fylke,nve_vatn_nr,utm_n,utm_e,utm_zone,lake_area_km2,altitude_m,comment_fm,comment_niva
0,1,2999.0,101-4-1,Femsjøen,Femsjøen,101,Halden kommune,1,Østfold,316.0,6.559050e+06,642460.000000,32,10.640000,79,OK - Overvåkes årlig for eutrofiparametere,NaN
1,2,9.0,101-2-7,Hokksjøen,Hokksjøen,101,Halden kommune,1,Østfold,3608.0,6.543369e+06,647241.906048,32,0.133879,148,OK,NaN
2,3,2998.0,101-2-2,Steinsvatnet,Steinsvatnet,101,Halden kommune,1,Østfold,3562.0,6.554004e+06,652477.000000,32,0.210000,178,OK,NaN
3,4,3000.0,105-3-6,Isesjøen,Isesjøen,105,Sarpsborg kommune,1,Østfold,133.0,6.572400e+06,626400.000000,32,6.200000,38,OK - Overvåkes årlig for eutrofiparametere,NaN
4,5,3001.0,105-3-10,Tunevatnet,Tunevatnet,105,Sarpsborg kommune,1,Østfold,3451.0,6.576676e+06,619486.000000,32,2.250000,40,OK - Overvåkes årlig for eutrofiparametere,NaN


In [4]:
# Convert co-ords
df = nivapy.spatial.utm_to_wgs84_dd(df, 
                                    zone='utm_zone',
                                    east='utm_e',
                                    north='utm_n')

df.head()

,index,station_id,station_code,station_name,lake_name,komm_nr,komm,fylke_nr,fylke,nve_vatn_nr,utm_n,utm_e,utm_zone,lake_area_km2,altitude_m,comment_fm,comment_niva,lat,lon
0,1,2999.0,101-4-1,Femsjøen,Femsjøen,101,Halden kommune,1,Østfold,316.0,6.559050e+06,642460.000000,32,10.640000,79,OK - Overvåkes årlig for eutrofiparametere,NaN,59.146735,11.490597
1,2,9.0,101-2-7,Hokksjøen,Hokksjøen,101,Halden kommune,1,Østfold,3608.0,6.543369e+06,647241.906048,32,0.133879,148,OK,NaN,59.004421,11.563584
2,3,2998.0,101-2-2,Steinsvatnet,Steinsvatnet,101,Halden kommune,1,Østfold,3562.0,6.554004e+06,652477.000000,32,0.210000,178,OK,NaN,59.097995,11.661994
3,4,3000.0,105-3-6,Isesjøen,Isesjøen,105,Sarpsborg kommune,1,Østfold,133.0,6.572400e+06,626400.000000,32,6.200000,38,OK - Overvåkes årlig for eutrofiparametere,NaN,59.271611,11.217840
4,5,3001.0,105-3-10,Tunevatnet,Tunevatnet,105,Sarpsborg kommune,1,Østfold,3451.0,6.576676e+06,619486.000000,32,2.250000,40,OK - Overvåkes årlig for eutrofiparametere,NaN,59.311994,11.098985


In [5]:
def html_text(row):
    station_id = row['station_id']
    station_code = row['station_code']
    station_name = row['station_name']
    komm = row['komm']
    komm_nr = int(row['komm_nr'])
    fylke = row['fylke']
    fylke_nr = int(row['fylke_nr'])
    nve_vatn_nr = row['nve_vatn_nr']
    lake_area_km2 = row['lake_area_km2']
    altitude = row['altitude_m']
    
    html = f"""
    <center><table>
      <tr>
        <td><b>Station ID:</b></td>
        <td>{station_id:.0f}</td>
      </tr>
      <tr>
        <td><b>Station code:</b></td>
        <td>{station_code}</td>
      </tr>
      <tr>
        <td><b>Station name:</b></td>
        <td>{station_name}</td>
      </tr>
      <tr>
        <td><b>Kommune:</b></td>
        <td>{komm} ({komm_nr})</td>
      </tr>
      <tr>
        <td><b>Fylke:</b></td>
        <td>{fylke} ({fylke_nr})</td>
      </tr>
      <tr>
        <td><b>NVE vatn nr.:</b></td>
        <td>{nve_vatn_nr:.0f}</td>
      </tr>
      <tr>
        <td><b>Area (km2):</b></td>
        <td>{lake_area_km2:.3f}</td>
      </tr>
      <tr>
        <td><b>Altitude (m):</b></td>
        <td>{altitude:.1f}</td>
      </tr>
    </table></center>
    """
    
    return html

df['html_text'] = df.apply(html_text, axis=1)

df.head()

,index,station_id,station_code,station_name,lake_name,komm_nr,komm,fylke_nr,fylke,nve_vatn_nr,utm_n,utm_e,utm_zone,lake_area_km2,altitude_m,comment_fm,comment_niva,lat,lon,html_text
0,1,2999.0,101-4-1,Femsjøen,Femsjøen,101,Halden kommune,1,Østfold,316.0,6.559050e+06,642460.000000,32,10.640000,79,OK - Overvåkes årlig for eutrofiparametere,NaN,59.146735,11.490597,\n <center><table>\n <tr>\n <td...
1,2,9.0,101-2-7,Hokksjøen,Hokksjøen,101,Halden kommune,1,Østfold,3608.0,6.543369e+06,647241.906048,32,0.133879,148,OK,NaN,59.004421,11.563584,\n <center><table>\n <tr>\n <td...
2,3,2998.0,101-2-2,Steinsvatnet,Steinsvatnet,101,Halden kommune,1,Østfold,3562.0,6.554004e+06,652477.000000,32,0.210000,178,OK,NaN,59.097995,11.661994,\n <center><table>\n <tr>\n <td...
3,4,3000.0,105-3-6,Isesjøen,Isesjøen,105,Sarpsborg kommune,1,Østfold,133.0,6.572400e+06,626400.000000,32,6.200000,38,OK - Overvåkes årlig for eutrofiparametere,NaN,59.271611,11.217840,\n <center><table>\n <tr>\n <td...
4,5,3001.0,105-3-10,Tunevatnet,Tunevatnet,105,Sarpsborg kommune,1,Østfold,3451.0,6.576676e+06,619486.000000,32,2.250000,40,OK - Overvåkes årlig for eutrofiparametere,NaN,59.311994,11.098985,\n <center><table>\n <tr>\n <td...


In [7]:
# Map
map1 = quickmap(df, 
                lon_col='lon', 
                lat_col='lat', 
                popup='html_text', 
                cluster=True, 
                tiles='OpenStreetMap', 
                aerial_imagery=True, 
                kartverket=True, 
                layer_name='Stations')
map1.save('index.html')
map1